# ADS 509 Sentiment Assignment

This notebook holds the Sentiment Assignment for Module 6 in ADS 509, Applied Text Mining. Work through this notebook, writing code and answering questions where required. 

In a previous assignment you put together Twitter data and lyrics data on two artists. In this assignment we apply sentiment analysis to those data sets. If, for some reason, you did not complete that previous assignment, data to use for this assignment can be found in the assignment materials section of Blackboard. 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [1]:
import os
import re
import emoji
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
from string import punctuation

from nltk.corpus import stopwords

sw = stopwords.words("english")

In [ ]:
# Add any additional import statements you need here


# Just in case I need something like this.
 
def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    return text.lower().strip().split() 


def remove_stop(tokens) :
    # modify this function to remove stopwords
    return([t for t in tokens if t.lower() not in sw])

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)




# apply the `pipeline` techniques from BTAP Ch 1 or 5

my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

lyrics_data["tokens"] = lyrics_data["lyrics"].apply(prepare,pipeline=my_pipeline)
lyrics_data["num_tokens"] = lyrics_data["tokens"].map(len) 


twitter_data["tokens"] = twitter_data["description"].apply(prepare,pipeline=my_pipeline)
twitter_data["num_tokens"] = twitter_data["tokens"].map(len) 

In [4]:
# change `data_location` to the location of the folder on your machine.
data_location = "C:/Users/zfreitas/Dropbox/Classes/USD/ADS-509-01-SP23 - Applied Text Mining/git/"

# These subfolders should still work if you correctly stored the 
# data from the Module 1 assignment
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

positive_words_file = "positive-words.txt"
negative_words_file = "negative-words.txt"
tidy_text_file = "tidytext_sentiments.txt"

## Data Input

Now read in each of the corpora. For the lyrics data, it may be convenient to store the entire contents of the file to make it easier to inspect the titles individually, as you'll do in the last part of the assignment. In the solution, I stored the lyrics data in a dictionary with two dimensions of keys: artist and song. The value was the file contents. A Pandas data frame would work equally well. 

For the Twitter data, we only need the description field for this assignment. Feel free all the descriptions read it into a data structure. In the solution, I stored the descriptions as a dictionary of lists, with the key being the artist. 




In [7]:
# Read in the lyrics data
lyrics_df = pd.read_pickle(data_location + "lyrics_df.pkl")
lyrics_df = lyrics_df[["title", "artist", "lyrics"]]
lyrics_df

,description,artist,clean_description,normalized_description,tokenized,tokens
0,,cher,,,[],[]
1,𝙿𝚛𝚘𝚞𝚍 𝚜𝚞𝚙𝚙𝚘𝚛𝚝𝚎𝚛 𝚘𝚏 𝚖𝚎𝚜𝚜𝚢 𝚋𝚞𝚗𝚜 & 𝚕𝚎𝚐𝚐𝚒𝚗𝚐𝚜,cher,𝙿𝚛𝚘𝚞𝚍 𝚜𝚞𝚙𝚙𝚘𝚛𝚝𝚎𝚛 𝚘𝚏 𝚖𝚎𝚜𝚜𝚢 𝚋𝚞𝚗𝚜 𝚕𝚎𝚐𝚐𝚒𝚗𝚐𝚜,Proud supporter of messy buns leggings,"[Proud, supporter, of, messy, buns, leggings]","[Proud, supporter, messy, buns, leggings]"
2,163㎝／愛かっぷ💜26歳🍒 工〇好きな女の子💓 フォローしてくれたらDMします🧡,cher,163㎝／愛かっぷ💜26歳🍒 工〇好きな女の子💓 フォローしてくれたらDMします🧡,163cm/愛かっふ💜26歳🍒 工〇好きな女の子💓 フォローしてくれたらDMします🧡,"[163cm, 愛かっふ, 26歳, 工〇好きな女の子, フォローしてくれたらDMします]","[163cm, 愛かっふ, 26歳, 工〇好きな女の子, フォローしてくれたらDMします]"
3,csu,cher,csu,csu,[csu],[csu]
4,Writer @Washinformer @SpelmanCollege alumna #D...,cher,Writer Washinformer SpelmanCollege alumna DCna...,Writer Washinformer SpelmanCollege alumna DCna...,"[Writer, Washinformer, SpelmanCollege, alumna,...","[Writer, Washinformer, SpelmanCollege, alumna,..."
...,...,...,...,...,...,...
8184476,"singer of songs, type 1 diabetic, tired $jakel...",robyn,singer of songs type 1 diabetic tired jakelgil...,singer of songs type 1 diabetic tired jakelgil...,"[singer, of, songs, type, diabetic, tired, jak...","[singer, songs, type, diabetic, tired, jakelgi..."
8184477,Dadx2/ Con-Arch/ Photographer/ DK #stemgrønnes...,robyn,Dadx2 ConArch Photographer DK stemgrønnest grø...,Dadx2 ConArch Photographer DK stemgrønnest grø...,"[Dadx2, ConArch, Photographer, DK, stemgrønnes...","[Dadx2, ConArch, Photographer, DK, stemgrønnes..."
8184478,A year to change a life is still a year ✨😌,robyn,A year to change a life is still a year ✨😌,A year to change a life is still a year ✨😌,"[A, year, to, change, a, life, is, still, a, y...","[year, change, life, still, year]"
8184479,Head of Consumer - Mango. Made in Melbourne. R...,robyn,Head of Consumer Mango Made in Melbourne Rambl...,Head of Consumer Mango Made in Melbourne Rambl...,"[Head, of, Consumer, Mango, Made, in, Melbourn...","[Head, Consumer, Mango, Made, Melbourne, Rambl..."


In [11]:
# Read in the twitter data
twitter_df = pd.read_pickle(data_location + "twitter_df.pkl")
twitter_df = twitter_df[["artist", "description"]]
twitter_df

,title,artist,lyrics
0,"""88 Degrees""",cher,"Stuck in L.A., ain't got no friends \nAnd so H..."
1,"""A Different Kind Of Love Song""",cher,What if the world was crazy and I was sane\nWo...
2,"""After All""",cher,"Well, here we are again\nI guess it must be fa..."
3,"""Again""",cher,Again evening finds me at your door \nHere to ...
4,"""Alfie""",cher,"What's it all about, Alfie?\nIs it just for th..."
...,...,...,...
714,"""We Dance To The Beat""",robyn,We dance to the beat\nWe dance to the beat\nWe...
715,"""Where Did Our Love Go""",robyn,Thoughts about you and me \nThinkin' about wha...
716,"""Who's That Girl""",robyn,Good girls are pretty like all the time\nI'm j...
717,"""With Every Heartbeat""",robyn,Maybe we could make it all right\nWe could mak...


In [ ]:
# Read in the positive and negative words and the
# tidytext sentiment. Store these so that the positive
# words are associated with a score of +1 and negative words
# are associated with a score of -1. You can use a dataframe or a 
# dictionary for this.

## Sentiment Analysis on Songs

In this section, score the sentiment for all the songs for both artists in your data set. Score the sentiment by manually calculating the sentiment using the combined lexicons provided in this repository. 

After you have calculated these sentiments, answer the questions at the end of this section.


In [ ]:
# your code here

### Questions

Q: Overall, which artist has the higher average sentiment per song? 

A: <!-- Your answer here -->

---

Q: For your first artist, what are the three songs that have the highest and lowest sentiments? Print the lyrics of those songs to the screen. What do you think is driving the sentiment score? 

A: <!-- Your answer here -->

---

Q: For your second artist, what are the three songs that have the highest and lowest sentiments? Print the lyrics of those songs to the screen. What do you think is driving the sentiment score? 

A: <!-- Your answer here -->

---

Q: Plot the distributions of the sentiment scores for both artists. You can use `seaborn` to plot densities or plot histograms in matplotlib.




## Sentiment Analysis on Twitter Descriptions

In this section, define two sets of emojis you designate as positive and negative. Make sure to have at least 10 emojis per set. You can learn about the most popular emojis on Twitter at [the emojitracker](https://emojitracker.com/). 

Associate your positive emojis with a score of +1, negative with -1. Score the average sentiment of your two artists based on the Twitter descriptions of their followers. The average sentiment can just be the total score divided by number of followers. You do not need to calculate sentiment on non-emoji content for this section.

In [ ]:
# your code here

Q: What is the average sentiment of your two artists? 

A: <!-- Your answer here --> 

---

Q: Which positive emoji is the most popular for each artist? Which negative emoji? 

A: <!-- Your answer here --> 

